In [ ]:
import cv2
import time
import os
import re
import pickle
import serial
import struct
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from IPython.display import clear_output
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.models import load_model

ser = serial.Serial('COM3', 57600, timeout=1)
images = []
labels = []

def calculateCheckSum(data):
    checksum = sum(data) % 256
    return checksum

def send_data(data):
    start_byte = b'\x55'
    end_byte = b'\xAA'
    packed_data = struct.pack('i',data)

    checksum = calculateCheckSum(packed_data)
    data_with_checksum = packed_data + bytes([checksum])
    
    packet = start_byte + data_with_checksum + end_byte
    print(packet)
    ser.write(packet)

def preprocessing(image_array):
    result = []
    for i in image_array:
        image = cv2.resize(i, (240, 240)) 
        image = image.astype('float') / 255 
        result.append(image)
    return np.array(result)

def neural():
    global model
    global encoder
    
    encoder = LabelBinarizer()
    encoder.fit(labels)
    print(encoder.transform(labels))

    
    base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (240, 240, 3))

    
    model = Sequential()
    
    model.add(base_model)
    
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(len(set(labels)), activation='softmax'))
    model.summary()

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer = Adam(), loss = "categorical_crossentropy", metrics = ['accuracy'])
    
    
    model.fit(images,encoder.transform(labels),epochs=31,batch_size=128,shuffle=True)

    score = model.evaluate(images,encoder.transform(labels))
    print('Test loss: ',score[0])
    print('Test accuracy: ',score[1])
    
    model.save('animal_model.h5')
    with open('animal_encoder.pkl', 'wb') as f:
        pickle.dump(encoder, f)
    model = load_model('animal_model.h5')
    with open('animal_encoder.pkl', 'rb') as f:
        e2 = pickle.load(f)

    model = load_model('animal_model.h5')
    with open('animal_encoder.pkl', 'rb') as f:
        encoder = pickle.load(f)

model = load_model('animal_model.h5')
with open('animal_encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

try:
    cap = cv2.VideoCapture(1)
    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    ret, img = cap.read()
    print('Start!')
    learn_val = 0
    while cap.isOpened():
        ret, img = cap.read()
        if img is None:
            print("Image Not Found")
            continue
        h, w = img.shape[:2]
        result_img = img.copy()
        cv2.imwrite('./temp.jpg',result_img)
        imgfile = './temp.JPG'
        img = cv2.imread(imgfile,cv2.IMREAD_COLOR)

        key = cv2.waitKey(1)
        result = -1
        if key == 27 or key == ord('q'):
            break
        clear_output(wait=True)

        result = model.predict(preprocessing([img]))
        print(result)
        val = encoder.inverse_transform(result)
        print(val)

        if(ser.in_waiting > 0):
            global request
            request = ser.read()
            print(request)
            if(request == b'\x01'):
                send_data(int(val))
                print("data sent")
            elif(request == b'\x02'):
                print('learning_start')
                learn_val = 1
        else:
            request = None
        
        if learn_val == 0:
            if(val == '1'):
                cv2.rectangle(img,(0,5),(200,45),(255,255,255),-1)
                cv2.putText(img,str("DOG (1)"),(5,35),0,1,(0,0,127),2)
                print("1: 강아지")
            elif(val == '2'):
                cv2.rectangle(img,(0,5),(200,45),(255,255,255),-1)
                cv2.putText(img,str("CAT (2)"),(5,35),0,1,(0,127,127),2)
                print("2: 고양이")
            elif(val == '3'):
                cv2.rectangle(img,(0,5),(200,45),(255,255,255),-1)
                cv2.putText(img,str("HORSE (3)"),(5,35),0,1,(0,127,127),2)
                print("3: 말")
            elif(val == '4'):
                cv2.rectangle(img,(0,5),(200,45),(255,255,255),-1)
                cv2.putText(img,str("COW (4)"),(5,35),0,1,(0,127,127),2)
                print("4: 소")
            elif(val == '5'):
                cv2.rectangle(img,(0,5),(200,45),(255,255,255),-1)
                cv2.putText(img,str("NOTHING (5)"),(5,35),0,1,(0,127,127),2)
                print("5: 없음")
            else:
                cv2.rectangle(img,(0,5),(200,45),(255,255,255),-1)
                cv2.putText(img,str("Not Define"),(5,35),0,1,(0,0,0),2)
                print("x: 미분류")
        elif learn_val == 1:
            if key == 27 or key == ord('q') or request == b'\x08':
                learn_val = 2
            elif key == ord('1') or request == b'\x03':
                cv2.imwrite('./animal_data/1_'+str(time.time()).replace('.','')+'.jpg',img)
            elif key == ord('2') or request == b'\x04':
                cv2.imwrite('./animal_data/2_'+str(time.time()).replace('.','')+'.jpg',img)
            elif key == ord('3') or request == b'\x05':
                cv2.imwrite('./animal_data/3_'+str(time.time()).replace('.','')+'.jpg',img)
            elif key == ord('4') or request == b'\x06':
                cv2.imwrite('./animal_data/4_'+str(time.time()).replace('.','')+'.jpg',img)
            elif key == ord('5') or request == b'\x07':
                cv2.imwrite('./animal_data/5_'+str(time.time()).replace('.','')+'.jpg',img)
        
        elif learn_val == 2:
            cv2.destroyAllWindows()
            
            for filename in os.listdir("animal_data"):  
                match = re.match("([0-9a-zA-Z]+)_.*", filename) 
                label = match.group(1)    
                labels.append(label)
                image = cv2.imread(os.path.join("./animal_data", filename), cv2.IMREAD_COLOR) 
                images.append(image) 
            images = preprocessing(images)
            neural()

            send_data(100)
            print("learning_end")
            time.sleep(1)
            while True:
                print(ser.in_waiting)
                if(ser.in_waiting > 0):
                    request = ser.read()
                    print(request)
                    if(request == b'\x01'):
                        send_data(100)
                        break
            learn_val = 0
            
        cv2.imshow('Animal Detect AI Program',img)

finally:
    if(ser.is_open):
        ser.close()
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 449ms/step
[[4.0835049e-04 1.7338012e-05 3.7756085e-06 3.1101517e-05 9.9953949e-01]]
['5']
b'\x01'
b'U\x05\x00\x00\x00\x05\xaa'
data sent
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]
 